# Symbol Table

This module contains the implementations of symbol tables

In [ ]:
#| default_exp symbol_table

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
#| output: false
from abc import ABC, abstractmethod
from typing import Iterable, Dict, Set, Callable, List, Union, Sequence, Tuple
from spannerlib.primitive_types import DataTypes, DataTypeMapping
from spannerlib.ie_function import IEFunction

Installation NLP failed


In [ ]:
#| export
class SymbolTableBase(ABC):
    """
    An abstraction for a symbol table. <br>
    the symbol table keeps track of: <br>
        1. the variables that were defined in the program, their types and their values <br> 
        2. the relations that were defined in the program and their schemas <br>
        3. the information extraction functions that were registered in the program and their data
    """

    @abstractmethod
    def set_var_value_and_type(self,
            var_name: str, # the name of the variable
            var_value: DataTypeMapping.term, # the value of the variable
            var_type: DataTypes # the type of the variable
            ) -> None:
        """ 
        Sets the type and value of a variable in the symbol table.
        """
        pass

    @abstractmethod
    def get_variable_type(self,
            var_name: DataTypeMapping.term # a variable name
            ) -> DataTypes: # the variable's type
        pass

    @abstractmethod
    def get_variable_value(self,
             var_name: DataTypeMapping.term # a variable name
             ) -> DataTypeMapping.term: # the variable's value
        pass

    @abstractmethod
    def get_all_variables(self) -> List[Tuple[str, DataTypes, DataTypeMapping.term]]:
        """
        @return: an iterable that contains tuples of the format (variable name, variable type, variable value)
        for each variable in the symbol table.
        """
        pass

    @abstractmethod
    def contains_variable(self,
                            var_name: str # a variable name
                            ) -> bool: # true if the variable is in the symbol table, else false
        pass

    @abstractmethod
    def add_relation_schema(self,
                            relation_name: str, # the relation's name.
                            schema: Sequence[DataTypes], # the relation's schema.
                            is_rule: bool # true if rule false if relation.
                            ) -> None:
        """
        Adds a new relation schema to the symbol table. <br>
        Note: Trying to add two schemas for the same relation will result in an exception as relation redefinitions are not allowed.
        """
        pass

    @abstractmethod
    def get_relation_schema(self, 
                            relation_name: str # a relation name
                            ) -> Sequence[DataTypes]: # the relation's schema
        pass

    @abstractmethod
    def get_all_relations(self) -> Sequence:
        """
        @return: an iterable that contains tuples of the format (relation name, relation schema)
        for each relation in the symbol table.
        """
        pass

    @abstractmethod
    def contains_relation(self,
                           relation_name: str # a relation name
                           ) -> bool: # true if the relation exists in the symbol table, else false
        pass

    @abstractmethod
    def register_ie_function(self, 
                             ie_function: Callable, 
                             ie_function_name: str, 
                             in_rel: Sequence[DataTypes],
                             out_rel: Union[List[DataTypes], Callable[[int], Sequence[DataTypes]]]
                             ) -> None:
        """
        Adds a new ie function to the symbol table.
        @see params in `IEFunction`'s __init__.
        """
        pass

    @abstractmethod
    def contains_ie_function(self,
                              ie_func_name: str # a name of an information extraction function
                              ) -> bool: # true if the ie function exists in the symbol table, else false
        pass

    @abstractmethod
    def get_ie_func_data(self, 
                         ie_func_name: str # a name of an information extraction function
                         ) -> IEFunction: # the ie function's data (see ie_function_base.IEFunctionData for more information on ie function data instances)
        pass

    @abstractmethod
    def get_all_registered_ie_funcs(self) -> Dict[str, IEFunction]:
        """
        @return: an iterable containing the names of all of the ie functions that are registered in the symbol table.
        """
        pass

    def register_predefined_ie_functions(self, 
                                         ie_funcs: Iterable[Dict] # iterable of the predefined ie functions in dict format
                                         ) -> None:
        """
        Adds to symbol table all the predefined ie functions.
        """
        for ie_func in ie_funcs:
            self.register_ie_function(**ie_func)

    @abstractmethod
    def remove_ie_function(self, 
                           name: str # the name of the ie function to remove
                           ) -> None:
        """
        Removes a function from the symbol table.
        """
        pass

    @abstractmethod
    def remove_all_ie_functions(self) -> None:
        """
        Removes all the ie functions from the symbol table.
        """
        pass

    @abstractmethod
    def print_registered_ie_functions(self) -> None:
        """
        Prints all the registered ie functions.
        """
        pass

    @abstractmethod
    def remove_rule_relation(self, 
                             relation_name: str # the name of the relation to remove
                             ) -> None:
        """
        Removes a rule relation from the symbol table.
        """
        pass

    @abstractmethod
    def remove_all_rule_relations(self) -> Iterable[str]:
        """
        Removes all the rule relations.

        @return: iterable of all the relations name it removed.
        """
        pass

    def __str__(self) -> str:
        """
        @return: a string representation of the symbol table for debugging purposes
        """

        # we will build the string incrementally using the string buffer
        string_buffer = list()

        # add the header of the variables table
        string_buffer.append('Variable\tType\tValue\n')
        # add the tuples of the variables table
        for name, var_type, var_value in self.get_all_variables():
            string_buffer.append(f'{name}\t{var_type}\t{var_value}\n')

        # add the header of the relation schemas table
        string_buffer.append('\nRelation\tSchema\n')
        # add the tuples of the relations schemas table
        for relation_name, type_list in self.get_all_relations():
            type_strings = [str(term_type) for term_type in type_list]
            type_list_string = ", ".join(type_strings)
            string_buffer.append(f"{relation_name}\t({type_list_string})\n")

        # add the header of the ie functions list
        string_buffer.append('\nregistered information extraction functions:\n')
        # add the ie functions
        for ie_func_name in self.get_all_registered_ie_funcs():
            string_buffer.append(f'{ie_func_name}\n')

        # combine the resulting tables to one string and return them
        symbol_table_string = ''.join(string_buffer)
        return symbol_table_string

In [ ]:
show_doc(SymbolTableBase.set_var_value_and_type)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L23){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.set_var_value_and_type

>      SymbolTableBase.set_var_value_and_type (var_name:str,
>                                              var_value:Union[str,spannerlog.primit
>                                              ive_types.Span,int], var_type:rgx
>                                              log.primitive_types.DataTypes)

Sets the type and value of a variable in the symbol table.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| var_name | str | the name of the variable |
| var_value | DataTypeMapping.term | the value of the variable |
| var_type | DataTypes | the type of the variable |
| **Returns** | **None** |  |

In [ ]:
show_doc(SymbolTableBase.get_variable_type)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L34){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_variable_type

>      SymbolTableBase.get_variable_type
>                                         (var_name:Union[str,spannerlog.primitive_t
>                                         ypes.Span,int])

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| var_name | DataTypeMapping.term | a variable name |
| **Returns** | **DataTypes** | **the variable's type** |

In [ ]:
show_doc(SymbolTableBase.get_variable_value)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L40){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_variable_value

>      SymbolTableBase.get_variable_value
>                                          (var_name:Union[str,spannerlog.primitive_
>                                          types.Span,int])

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| var_name | DataTypeMapping.term | a variable name |
| **Returns** | **DataTypeMapping.term** | **the variable's value** |

In [ ]:
show_doc(SymbolTableBase.get_all_variables)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L46){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_all_variables

>      SymbolTableBase.get_all_variables ()

@return: an iterable that contains tuples of the format (variable name, variable type, variable value)
for each variable in the symbol table.

In [ ]:
show_doc(SymbolTableBase.contains_variable)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L54){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.contains_variable

>      SymbolTableBase.contains_variable (var_name:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| var_name | str | a variable name |
| **Returns** | **bool** | **true if the variable is in the symbol table, else false** |

In [ ]:
show_doc(SymbolTableBase.add_relation_schema)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L60){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.add_relation_schema

>      SymbolTableBase.add_relation_schema (relation_name:str,
>                                           schema:Sequence[spannerlog.primitive_typ
>                                           es.DataTypes], is_rule:bool)

Adds a new relation schema to the symbol table. <br>
Note: Trying to add two schemas for the same relation will result in an exception as relation redefinitions are not allowed.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_name | str | the relation's name. |
| schema | Sequence[DataTypes] | the relation's schema. |
| is_rule | bool | true if rule false if relation. |
| **Returns** | **None** |  |

::: {.callout-note}

Trying to add two schemas for the same relation will result in an exception as relation redefinitions are not allowed.

:::

In [ ]:
show_doc(SymbolTableBase.get_relation_schema)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L72){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_relation_schema

>      SymbolTableBase.get_relation_schema (relation_name:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_name | str | a relation name |
| **Returns** | **Sequence[DataTypes]** | **the relation's schema** |

In [ ]:
show_doc(SymbolTableBase.get_all_relations)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L78){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_all_relations

>      SymbolTableBase.get_all_relations ()

@return: an iterable that contains tuples of the format (relation name, relation schema)
for each relation in the symbol table.

In [ ]:
show_doc(SymbolTableBase.contains_relation)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L86){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.contains_relation

>      SymbolTableBase.contains_relation (relation_name:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_name | str | a relation name |
| **Returns** | **bool** | **true if the relation exists in the symbol table, else false** |

In [ ]:
show_doc(SymbolTableBase.register_ie_function)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L92){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.register_ie_function

>      SymbolTableBase.register_ie_function (ie_function:Callable,
>                                            ie_function_name:str, in_rel:Sequen
>                                            ce[spannerlog.primitive_types.DataTypes
>                                            ], out_rel:Union[List[spannerlog.primit
>                                            ive_types.DataTypes],Callable[[int]
>                                            ,Sequence[spannerlog.primitive_types.Da
>                                            taTypes]]])

Adds a new ie function to the symbol table.
@see params in `IEFunction`'s __init__.

In [ ]:
show_doc(SymbolTableBase.contains_ie_function)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L105){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.contains_ie_function

>      SymbolTableBase.contains_ie_function (ie_func_name:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| ie_func_name | str | a name of an information extraction function |
| **Returns** | **bool** | **true if the ie function exists in the symbol table, else false** |

In [ ]:
show_doc(SymbolTableBase.get_ie_func_data)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L111){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_ie_func_data

>      SymbolTableBase.get_ie_func_data (ie_func_name:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| ie_func_name | str | a name of an information extraction function |
| **Returns** | **IEFunction** | **the ie function's data (see ie_function_base.IEFunctionData for more information on ie function data instances)** |

In [ ]:
show_doc(SymbolTableBase.get_all_registered_ie_funcs)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L117){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.get_all_registered_ie_funcs

>      SymbolTableBase.get_all_registered_ie_funcs ()

@return: an iterable containing the names of all of the ie functions that are registered in the symbol table.

In [ ]:
show_doc(SymbolTableBase.register_predefined_ie_functions)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L123){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.register_predefined_ie_functions

>      SymbolTableBase.register_predefined_ie_functions
>                                                        (ie_funcs:Iterable[Dict
>                                                        ])

Adds to symbol table all the predefined ie functions.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| ie_funcs | Iterable[Dict] | iterable of the predefined ie functions in dict format |
| **Returns** | **None** |  |

In [ ]:
show_doc(SymbolTableBase.remove_ie_function)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L133){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.remove_ie_function

>      SymbolTableBase.remove_ie_function (name:str)

Removes a function from the symbol table.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| name | str | the name of the ie function to remove |
| **Returns** | **None** |  |

In [ ]:
show_doc(SymbolTableBase.remove_all_ie_functions)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L142){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.remove_all_ie_functions

>      SymbolTableBase.remove_all_ie_functions ()

Removes all the ie functions from the symbol table.

In [ ]:
show_doc(SymbolTableBase.print_registered_ie_functions)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L149){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.print_registered_ie_functions

>      SymbolTableBase.print_registered_ie_functions ()

Prints all the registered ie functions.

In [ ]:
show_doc(SymbolTableBase.remove_rule_relation)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L156){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.remove_rule_relation

>      SymbolTableBase.remove_rule_relation (relation_name:str)

Removes a rule relation from the symbol table.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_name | str | the name of the relation to remove |
| **Returns** | **None** |  |

In [ ]:
show_doc(SymbolTableBase.remove_all_rule_relations)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/symbol_table.py#L165){target="_blank" style="float:right; font-size:smaller"}

### SymbolTableBase.remove_all_rule_relations

>      SymbolTableBase.remove_all_rule_relations ()

Removes all the rule relations.

@return: iterable of all the relations name it removed.

In [ ]:
#| export
#| hide
class SymbolTable(SymbolTableBase):
    def __init__(self) -> None:
        self._var_to_value: Dict[str, DataTypeMapping.term] = {}
        self._var_to_type: Dict[str, DataTypes] = {}
        self._relation_to_schema: Dict[str, Sequence[DataTypes]] = {}
        self._registered_ie_functions: Dict[str, IEFunction] = {}
        self._rule_relations: Set[str] = set()
        #TODO here and in base, add a dict of registered agg functions, mirror ie functions

    def set_var_value_and_type(self, var_name: str, var_value: DataTypeMapping.term, var_type: DataTypes) -> None:
        self._var_to_value[var_name] = var_value
        self._var_to_type[var_name] = var_type

    def get_variable_type(self, var_name: DataTypeMapping.term) -> DataTypes:
        return self._var_to_type[str(var_name)]

    def get_variable_value(self, var_name: DataTypeMapping.term) -> DataTypeMapping.term:
        return self._var_to_value[str(var_name)]

    def get_all_variables(self) -> List[Tuple[str, DataTypes, DataTypeMapping.term]]:
        all_vars = []
        for var_name in self._var_to_type.keys():
            var_type = self._var_to_type[var_name]
            var_value = self._var_to_value[var_name]
            all_vars.append((var_name, var_type, var_value))
        return all_vars

    def contains_variable(self, var_name: str) -> bool:
        return var_name in self._var_to_type

    def add_relation_schema(self, relation_name: str, schema: Sequence[DataTypes], is_rule: bool) -> None:
        # rule can be defined multiple times with same head (unlike relation)
        if is_rule:
            err_msg = f'relation "{relation_name}" already has a different schema'
            # if the rule is already defined the current schema must be equal to the rule's schema
            if relation_name in self._relation_to_schema:
                # check that relation name was actually defined as a rule and not as a relation
                if relation_name not in self._rule_relations:
                    raise Exception(err_msg)
                if not self._relation_to_schema[relation_name] == schema:
                    raise Exception(err_msg)
                return

        else:  # relation definition
            # if the relation is already defined we can't redefine her
            if relation_name in self._relation_to_schema:
                raise Exception(f'relation "{relation_name}" already has a schema')

        self._relation_to_schema[relation_name] = schema
        if is_rule:
            self._rule_relations.add(relation_name)

    def get_relation_schema(self, relation_name: str) -> Sequence[DataTypes]:
        if relation_name not in self._relation_to_schema:
            raise NameError(f"relation {relation_name} does not exist in the symbol table")
        return self._relation_to_schema[relation_name]

    def get_all_relations(self) -> Sequence:
        return [(relation, schema) for relation, schema in self._relation_to_schema.items()]

    def contains_relation(self, relation_name: str) -> bool:
        return relation_name in self._relation_to_schema

    def register_ie_function(self, ie_function: Callable, ie_function_name: str, in_rel: Sequence[DataTypes],
                             out_rel: Union[List[DataTypes], Callable[[int], Sequence[DataTypes]]]) -> None:
        self._registered_ie_functions[ie_function_name] = IEFunction(ie_function, in_rel, out_rel)

    def register_ie_function_object(self, ie_function_object: IEFunction, ie_function_name: str) -> None:
        self._registered_ie_functions[ie_function_name] = ie_function_object

    def contains_ie_function(self, ie_func_name: str) -> bool:
        return ie_func_name in self._registered_ie_functions

    def get_ie_func_data(self, ie_func_name: str) -> IEFunction:
        if self.contains_ie_function(ie_func_name):
            return self._registered_ie_functions[ie_func_name]
        else:
            raise ValueError(f"'{ie_func_name}' is not a registered function.")

    def get_all_registered_ie_funcs(self) -> Dict[str, IEFunction]:
        #return deepcopy(self._registered_ie_functions)
        return self._registered_ie_functions.copy()
        
    def remove_ie_function(self, name: str) -> None:
        if not self._registered_ie_functions.pop(name, None):
            raise ValueError(f"IE function named {name} doesn't exist")

    def remove_all_ie_functions(self) -> None:
        self._registered_ie_functions.clear()

    def print_registered_ie_functions(self) -> None:
        for ie_function_name, ie_function_obj in self._registered_ie_functions.items():
            print(f'{ie_function_name}\n{ie_function_obj.get_meta_data}\n{ie_function_obj}\n'
                  f'{ie_function_obj.ie_function_def.__doc__}\n\n')

    def remove_rule_relation(self, relation_name: str) -> None:
        try:
            self._rule_relations.remove(relation_name)
            del self._relation_to_schema[relation_name]
        except KeyError:
            raise KeyError(f"An attempt to delete unfound relation {relation_name}")

    def remove_all_rule_relations(self) -> Set[str]:
        relations_names = self._rule_relations

        self._relation_to_schema = {relation: schema for relation, schema in self._relation_to_schema.items()
                                    if relation not in relations_names}
        self._rule_relations = set()
        return relations_names


::: {.callout-note collapse="true"}

##### Example

In [ ]:
table = SymbolTable()
table.set_var_value_and_type("var1", 10, int)
print("var1 type is",table.get_variable_type("var1"),"and its value is",table.get_variable_value("var1"))

var1 type is <class 'int'> and its value is 10


In [ ]:
table.add_relation_schema("relation1", [int, str], is_rule=False)
print(table.get_relation_schema("relation1"))

[<class 'int'>, <class 'str'>]


In [ ]:
def ie_function(x):
        return x + 1
table.register_ie_function(ie_function, "func1", [DataTypes.integer], [DataTypes.integer])
print(("func1" in table.get_all_registered_ie_funcs()))

True


:::

#| hide
##### TEST

In [ ]:
#| hide
import pytest

In [ ]:
#| hide
#test_variable_operations
table = SymbolTable()
table.set_var_value_and_type("var1", 10, int)
assert table.contains_variable("var1")
assert table.get_variable_type("var1") == int
assert table.get_variable_value("var1") == 10

In [ ]:
#| hide
# test_relation_operations
table = SymbolTable()
table.add_relation_schema("relation1", [int, str], is_rule=False)
assert table.contains_relation("relation1")
assert table.get_relation_schema("relation1") == [int, str]

In [ ]:
#| hide
#test_variable_relation_interactions
table = SymbolTable()

table.set_var_value_and_type("var1", 10, DataTypes.integer)
assert table.contains_variable("var1")
assert table.get_variable_type("var1") == DataTypes.integer
assert table.get_variable_value("var1") == 10

table.add_relation_schema("relation1", [DataTypes.integer, DataTypes.string], is_rule=False)
assert table.contains_relation("relation1")
assert table.get_relation_schema("relation1") == [DataTypes.integer, DataTypes.string]

with pytest.raises(KeyError):
    table.get_variable_type("relation1")
with pytest.raises(NameError):
    table.get_relation_schema("var1")

In [ ]:
#| hide
#test_ie_function_operations
def ie_function(x):
    return x + 1

table = SymbolTable()
table.register_ie_function(ie_function, "func1", [DataTypes.integer], [DataTypes.integer])

assert table.contains_ie_function("func1")
assert table.get_ie_func_data("func1").ie_function_def(5) == 6

table.remove_ie_function("func1")
assert not table.contains_ie_function("func1")

In [ ]:
#| hide
#test_rule_relation_operations
table = SymbolTable()

table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)
assert table.contains_relation("rule_relation1")
assert "rule_relation1" in [relation for relation, _ in table.get_all_relations()]

table.remove_rule_relation("rule_relation1")
assert not table.contains_relation("rule_relation1")
assert "rule_relation1" not in [relation for relation, _ in table.get_all_relations()]

In [ ]:
#| hide
#test_all_relations_and_ie_functions
table = SymbolTable()

table.add_relation_schema("relation1", [DataTypes.integer], is_rule=False)
table.add_relation_schema("relation2", [DataTypes.string], is_rule=False)

def ie_function(x):
    return x + 1

table.register_ie_function(ie_function, "func1", [DataTypes.integer], [DataTypes.integer])
table.register_ie_function(ie_function, "func2", [DataTypes.string], [DataTypes.string])

all_relations = table.get_all_relations()
assert ("relation1", [DataTypes.integer]) in all_relations
assert ("relation2", [DataTypes.string]) in all_relations

all_ie_functions = table.get_all_registered_ie_funcs()
assert "func1" in all_ie_functions
assert "func2" in all_ie_functions

In [ ]:
#| hide
#test_rule_relation_schema_check
table = SymbolTable()

table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)

with pytest.raises(Exception):
    table.add_relation_schema("rule_relation1", [DataTypes.integer, DataTypes.string], is_rule=True)

In [ ]:
#| hide
#test_rule_relation_name_conflict
table = SymbolTable()

table.add_relation_schema("relation1", [DataTypes.integer], is_rule=False)

with pytest.raises(Exception):
    table.add_relation_schema("relation1", [DataTypes.string, DataTypes.integer], is_rule=True)

In [ ]:
#| hide
#test_remove_rule_relation_bug
table = SymbolTable()

table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)

table.remove_rule_relation("rule_relation1")

with pytest.raises(KeyError):
    table.remove_rule_relation("rule_relation1")